# **Install MMDetection**

In [ ]:
# All you need to do is upload your configuration file and model file!


In [ ]:
from glob import glob
test_dir = glob('/kaggle/input/sartorius-cell-instance-segmentation/test/*')
if len(test_dir) != 3:
    debug = False
else:
    debug = True
print(f"debug={debug}")

In [ ]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

In [ ]:
!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!rm -rf mmdetection

!cp -r ../input/edited-mmdetection /kaggle/working/
!mv /kaggle/working/edited-mmdetection /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e .

# **Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cupy as cp
import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
import json
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import numpy.random
import random
import cv2
import re
import shutil
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed
from glob import glob
from tqdm.notebook import tqdm

In [ ]:
%cd ..

# **Helper Functions**

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))
def get_mask_from_result(result):
    d = {True : 1, False : 0}
    u,inv = np.unique(result,return_inverse = True)
    mk = cp.array([d[x] for x in u])[inv].reshape(result.shape)
#     print(mk.shape)
    return mk
def does_overlap(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            return True
    return False


def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            print("Overlap detected")
            mask[np.logical_and(mask, other_mask)] = 0
    return mask
def get_img_and_mask(img_path, annotation, width, height):
    """ Capture the relevant image array as well as the image mask """
    img_mask = np.zeros((height, width), dtype=np.uint8)
    for i, annot in enumerate(annotation): 
        img_mask = np.where(rle_decode(annot, (height, width))!=0, i, img_mask)
    img = cv2.imread(img_path)[..., ::-1]
    return img[..., 0], img_mask

def plot_img_and_mask(img, mask, invert_img=True, boost_contrast=True):
    """ Function to take an image and the corresponding mask and plot
    
    Args:
        img (np.arr): 1 channel np arr representing the image of cellular structures
        mask (np.arr): 1 channel np arr representing the instance masks (incrementing by one)
        invert_img (bool, optional): Whether or not to invert the base image
        boost_contrast (bool, optional): Whether or not to boost contrast of the base image
        
    Returns:
        None; Plots the two arrays and overlays them to create a merged image
    """
    plt.figure(figsize=(20,10))
    
    plt.subplot(1,3,1)
    _img = np.tile(np.expand_dims(img, axis=-1), 3)
    
    # Flip black-->white ... white-->black
    if invert_img:
        _img = _img.max()-_img
        
    if boost_contrast:
        _img = np.asarray(ImageEnhance.Contrast(Image.fromarray(_img)).enhance(16))
        
    plt.imshow(_img)
    plt.axis(False)
    plt.title("Cell Image", fontweight="bold")
    
    plt.subplot(1,3,2)
    _mask = np.zeros_like(_img)
    _mask[..., 0] = mask
    plt.imshow(mask, cmap='rainbow')
    plt.axis(False)
    plt.title("Instance Segmentation Mask", fontweight="bold")
    
    merged = cv2.addWeighted(_img, 0.75, np.clip(_mask, 0, 1)*255, 0.25, 0.0,)
    plt.subplot(1,3,3)
    plt.imshow(merged)
    plt.axis(False)
    plt.title("Cell Image w/ Instance Segmentation Mask Overlay", fontweight="bold")
    
    plt.tight_layout()
    plt.show()

# **Model**

In [ ]:
config_file = '../input/cell-pt/job5000_cascade_mask_rcnn_x101_64x4d_fpn_20e_coco.py'
checkpoint_file = '../input/cell-pt/adam/epoch_20.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [ ]:
masks = []
files = []
confidence_thresholds = {0: 0.25, 1: 0.55, 2: 0.65}
for imgs in tqdm(glob('../input/sartorius-cell-instance-segmentation/test/*')):
    result = inference_detector(model, imgs)
    pred_class_ls = [len(result[0][0]), len(result[0][1]), len(result[0][2])]
    pred_class = pred_class_ls.index(max(len(result[0][0]), len(result[0][1]), len(result[0][2])))
    if debug:
        img = mmcv.imread(imgs)
        show_result_pyplot(model, img, result)
    msk = []
    for i, classe in enumerate(result[0]):
        if classe.shape != (0, 5):
            bbs = classe
            sgs = result[1][i]
            for bb, sg in zip(bbs,sgs):
                box = bb[:4]
                cnf = bb[4]
                if cnf >= confidence_thresholds[pred_class]:
                    mask = get_mask_from_result(sg)
                    mask = remove_overlapping_pixels(mask, msk)
                    msk.append(mask)
    for mk in msk:
            rle_mask = rle_encoding(mk)
            masks.append(rle_mask)
            files.append(str(imgs.split('/')[4].split('.')[0]))

In [ ]:
files = pd.Series(files, name='id')
preds = pd.Series(masks, name='predicted')
submission_df = pd.concat([files, preds], axis=1)
submission_df.to_csv('submission.csv', index=False)
submission_df

In [ ]:
%cd /kaggle/working
!rm -rf /kaggle/working/mmpycocotools-12.0.3
!rm -rf /kaggle/working/addict-2.4.0-py3-none-any.whl
!rm -rf /kaggle/working/pycocotools-2.0.2
!rm -rf /kaggle/working/cascade_rcnn_r2_101_fpn_20e_coco.py
!rm -rf /kaggle/working/mmcv-1.3.8
!rm -rf /kaggle/working/terminal-0.4.0-py3-none-any.whl
!rm -rf /kaggle/working/mmcv_full-1_3_8-cu110-torch1_7_0
!rm -rf /kaggle/working/terminaltables-3.1.0-py3-none-any.whl
!rm -rf /kaggle/working/mmdetection
!rm -rf /kaggle/working/yapf-0.31.0-py2.py3-none-any.whl
!rm -rf /kaggle/working/cascade_mask_rcnn_x101_64x4d_fpn_20e_coco.py